In [1]:
from selenium import webdriver
import time, pandas as pd, numpy as np, os
from selenium.webdriver.common.by import By


from dotenv import load_dotenv

load_dotenv()


local = os.environ.get("RUN")
print(local)

from local import options


MAIN_URL  = 'https://cuantoestaeldolar.pe'


if local == 'local':
    driver = webdriver.Chrome()
else:
    driver = webdriver.Chrome(options=options)

driver.get(MAIN_URL)

# sunat

a = driver.find_element(By.XPATH, '//*[@id="converter"]/div/div[2]/div[1]')

print(a.text)
# firsm

local
Sunat
3.726
0.00
3.732
0.00


In [116]:
import numpy as np
import itertools

In [121]:
div_elements = driver.find_elements(By.CSS_SELECTOR, "div.flex.my-auto a")

# Itera sobre los elementos encontrados y extrae el atributo href de cada uno
hrefs = [element.get_attribute("href") for element in div_elements]
# hrefs

In [43]:
'//*[@id="__next"]/main/div[3]/div[6]/div/div[1]/div[29]'
a ='//*[@id="__next"]/main/div[3]/div[6]/div/div[1]/div[29]/div[1]'
driver.find_element(By.XPATH, a).text.split('\n')

['24 min', 'Cambiar', '3.730', '0.00', '3.742', '0.00']

In [15]:
texto_numerico_element = driver.find_element(By.CSS_SELECTOR, 'p.ValueQuotation_text___mR_0')
texto_numerico = texto_numerico_element.text.strip()
texto_numerico_element.text

''

In [214]:
# left div
N_rows = 13
MIN_LEFT = 29
MIN_RIGHT = 2
COLUMNS_NAMES = ["actualizacion", "trash1", "compra", 'variacion_compra','venta', 'variacion_venta', 'img_url', 'companny_url']

# Companny
range_left = list(range(MIN_LEFT, (MIN_LEFT + N_rows + 1)))
range_right = list(range(MIN_RIGHT, (MIN_RIGHT + N_rows + 1)))



PREV_XPATH = '//*[@id="__next"]/main/div[3]/'

def gen_divs(n, left = True, prev_ndiv=PREV_XPATH, n_div_tbl_location = 6, after = 'div[1]'):
    values_divs = range_right
    div_col = 2
    if left:
        values_divs = range_left
        div_col = 1
    
    n_value = values_divs[n]

    div_location = f"div[{n_div_tbl_location}]/div/"
    div_column = f"div[{div_col}]/"
    div_row = f"div[{n_value}]/{after}"
    div = prev_ndiv + div_location + div_column + div_row
    return div

def gen_img(div, after = "/div[1]/div[1]/span/img", bank = False):
    if bank:
        after = '/div[1]/span/img'

    return div + after

def gen_href(div, after = "/div[3]/a"):
    return div + after

def value_element_xpath(xpath, driver=driver):
    return driver.find_element(By.XPATH, xpath)

def gen_dolar_values(div):
    try:
      dolar = value_element_xpath(div).text.split("\n")
    except:
      dolar = ["dummy", "Cambiar", np.nan, np.nan, np.nan, np.nan]
    return dolar
def gen_url_imagen(div):
    return value_element_xpath(div).get_attribute('src')
def gen_url_companny(div):
    return value_element_xpath(div).get_attribute('href')

def get_values_dolar(srt_values):
    base, img, companny = srt_values
    dolar = gen_dolar_values(base)
    img_url = gen_url_imagen(img)
    companny_url = gen_url_companny(companny)

    return dolar + [img_url, companny_url]

def xpaths_divs(total = N_rows):
    left_right = [True, False]
    combinations = list(itertools.product(list(range(total)), left_right))
    xpaths_base = [gen_divs(n_row, col) for n_row, col in combinations]
    xpath_img = [gen_img(xpath) for xpath in xpaths_base]
    xpath_a = [gen_href(xpath) for xpath in xpaths_base]

    sorted_xpath = [[base, img, a] for base, img, a in zip(xpaths_base, xpath_img, xpath_a)]

    return sorted_xpath


def extract_values():
    xpaths = xpaths_divs()
    result = [get_values_dolar(xpath) for xpath in xpaths]
    # return result
    return pd.DataFrame(result)
# Extract
data_dolar = extract_values()
data_dolar.columns = COLUMNS_NAMES
data_dolar


,actualizacion,trash1,compra,variacion_compra,venta,variacion_venta,img_url,companny_url
0,2 horas,Cambiar,3.730,0.00,3.742,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://dollarhouse.pe/?utm_source=ced
1,15 min,Cambiar,3.717,0.00,3.747,0.00,https://cuantoestaeldolar.pe/_next/image?url=h...,https://web.rextie.com/registrate?utm_source=c...
2,2 horas,Cambiar,3.723,0.00,3.747,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://cambiafx.pe/?utm_source=display&utm_me...
3,2 horas,Cambiar,3.732,0.00,3.739,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://www.cambiomundial.com/?utm_source=ced
4,2 horas,Cambiar,3.726,-0.004,3.739,-0.001,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://mpodera.pe/?utm_source=ced
5,2 horas,Cambiar,3.710,-0.023,3.760,0.015,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://www.okanecambiodigital.com/?utm_source...
6,1 horas,Cambiar,3.731,-0.002,3.749,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://dichikash.com/?utm_source=ced
7,2 horas,Cambiar,3.724,-0.002,3.750,0.002,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://chapacambio.com/?c=CHAPACUANTO
8,2 horas,Cambiar,3.732,0.00,3.753,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://westernunionperu.pe/cambiodemoneda?utm...
9,3 horas,Cambiar,3.728,-0.002,3.740,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://dolarex.pe/?utm_source=ced


In [171]:


import re
def extract_domain(url):
    match = re.search(r'https?://(www\.)?([^/]+)', url)
    if match:
        domain = match.group(2).split('.')[0]
        return domain
    return None
data_dolar['companny_name'] = data_dolar['companny_url'].apply(extract_domain)
data_dolar.head()


,actualizacion,trash1,compra,variacion_compra,venta,variacion_venta,img_url,companny_url,companny_name
0,1 horas,Cambiar,3.730,0.00,3.742,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://dollarhouse.pe/?utm_source=ced,dollarhouse
1,11 min,Cambiar,3.712,0.00,3.747,0.00,https://cuantoestaeldolar.pe/_next/image?url=h...,https://web.rextie.com/registrate?utm_source=c...,web
2,1 horas,Cambiar,3.723,0.00,3.747,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://cambiafx.pe/?utm_source=display&utm_me...,cambiafx
3,1 horas,Cambiar,3.732,0.00,3.739,0.00,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://www.cambiomundial.com/?utm_source=ced,cambiomundial
4,1 horas,Cambiar,3.726,-0.004,3.739,-0.001,https://s3-ced-uploads-01.s3.amazonaws.com/170...,https://mpodera.pe/?utm_source=ced,mpodera


In [212]:
N_banks = 5
PREV_XPATH
def gen_divs_bank(n = N_banks):
    divs_bank = [PREV_XPATH + f"div[9]/div/div[2]/div[{_n}]/div[1]" for _n in range(1, n+1)]
    return divs_bank
def bank_name(url):
    try:
        parts = url.split("/")
        name = parts[-1].split(".")[0]
    except:
        name = "None"
    return name


banks_xpaths = gen_divs_bank()
dolar_values = [gen_dolar_values(bank) for bank in banks_xpaths]

bank_imgs = [gen_url_imagen(gen_img(bank, bank=True)) for bank in banks_xpaths]
banks_data = [dolar + [img] for dolar, img in zip(dolar_values, bank_imgs)]
banks_data = pd.DataFrame(banks_data, columns=['compra', 'venta', 'img_url'])
banks_data['companny_name'] = banks_data['img_url'].apply(bank_name)
banks_data

,compra,venta,img_url,companny_name
0,3.650,3.825,https://cuantoestaeldolar.pe/icons/scotiabank.svg,scotiabank
1,3.693,3.780,https://cuantoestaeldolar.pe/icons/bcp.svg,bcp
2,3.682,3.805,https://cuantoestaeldolar.pe/icons/interbank.svg,interbank
3,3.690,3.760,https://cuantoestaeldolar.pe/icons/bn.svg,bn
4,3.638,3.820,https://cuantoestaeldolar.pe/icons/bbva.svg,bbva
